In [5]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict

In [30]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file
def checkDir(dirLoc):
    print
    if os.path.exists(dirLoc):#Checks if the dir exists
        print("The directory exists")
    else:
        print ("No directory found for "+dirLoc) #Output if no directory
#         print
#         os.makedirs(CheckDir)#Creates a new dir for the given name
#         print "Directory created for "+CheckDir

In [96]:
folderList

['FinDataReal', 'FinData', 'Data']

In [24]:
dataOnlyDrive = '/media/ak/DataOnly'  # external date only drive
#ext_drive_loc = ('/media/ak/My Passport/Experiment Data/')
data_dir = os.getenv('FINANCE_DATA') #internal folder with finance data 
folderList =  [s for s in os.listdir(dataOnlyDrive) if s.startswith('Dat') or s.startswith('Fin')]
# inputDrive = data_only_drive
# features = os.path.join(inputDrive,'features')
# labels = os.path.join(inputDrive, 'labels')

# symbols = [s for s in os.listdir(features) if s.endswith('.L')]
folderIdx = 0
finalLocation= "/".join((dataOnlyDrive,folderList[folderIdx]))
symbols = [s for s in os.listdir(finalLocation) if s.endswith('.L')] #keep a list of the symbols
os.listdir(finalLocation)

['CNA.L', 'PRU.L', 'Labels', 'REL.L', 'BARC.L']

## pick a symbol ## 

In [107]:
##picking up a specific feature
symbolIdx = 1 #pick one of the symbols
#symbols[symbolIdx] -->output :PRU.L

# do a join to get the location
symbolLocation = "/".join((finalLocation,symbols[symbolIdx])) 

# get he features now
symbolFeaturesLocation = "/".join((symbolLocation,'MODEL_BASED')) # where all the HMM output is

print(symbolFeaturesLocation) # <-- all the HMM model output is here, for each model there is a Date Folder and then OOS files

/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED


## pick Features for a specific Dates ## 

In [103]:

symbolFeaturesDatesList = os.listdir(symbolFeaturesLocation) #list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
## (symbolFeaturesDatesList = '20170829', '20170710', '20170801', ... ]


### now lets move to labels ...pick the location of the labels and make a list of all the available labels ##

In [106]:
#location of labels : /media/ak/DataOnly/FinDataReal/Labels/[Symbol :PRU.L]/NON_DIRECTIONAL

symbolLabelsLocation = "/".join((finalLocation, 'Labels',symbols[symbolIdx],'NON_DIRECTIONAL')) 

#list of all the label dates

symbolLabelsDates =[dateFile.split(".")[0] for dateFile in os.listdir(symbolLabelsLocation)]

# Output symbolLabelsDates --> ['20170704', '20180226', '20180208',...] all we are doing is going for this 


/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL


In [146]:
##now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
featrsIdx = 3 #symbol-hmm-model-date index

#this symbolFeaturesDatesList[featrsIdx] will give you a date: 20170710 =which contains all the HMM induced featureb

symbolOneFeaturesDate = "/".join((symbolFeaturesLocation, symbolFeaturesDatesList[featrsIdx]))
# output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710

symbolEachModelFeaturesDates=[file.split("_")[5] for file in os.listdir(symbolOneFeaturesDate )] 

print("you are picking this HMM model date:",symbolFeaturesDatesList[featrsIdx])

you are picking this HMM model date: 20170120


### method to find the list of dates (intersection) that we have both HMM-model-features and labels


In [147]:

a = symbolEachModelFeaturesDates # set of OutOfSample Dates produced by each HMM Model
b = symbolLabelsDates # set of all the Labels 

# we want to match OOS Dates with Labels 
commonDates =list(set(a) & set(b))

# for these dates we have features and we have labels- so we can fit and predict! this produces a list of [date1, date 2, ...]

In [114]:
#Now we need to reconstruct the dates for the fitting
commonIdx =0 #common dates index
commonDates[commonIdx] # common features/labels- so pick a day with idx

print(os.listdir(symbolOneFeaturesDate)[2]) #we picked one file
# symbolFeaturesDatesList[featrsIdx] 
# the cache of the file includes the date it was saved, the first date is the Common Date and the next date is the "creation date"
createDate = os.listdir(symbolOneFeaturesDate)[2].split("_")[7] #stupid hack
print(createDate)

PRU.L_3_states_features_date:_20170728_now:_20181229_.pickle
20181229


## now we reconstruct the file but with arbitrary new date ##

In [116]:
commonDatesFeatureDateFile = \
"".join((symbols[1],"_3_states_features_date:_",commonDates[commonIdx], \
         "_now:_",createDate,"_.pickle"))
print(commonDatesFeatureDateFile) 

PRU.L_3_states_features_date:_20170726_now:_20181229_.pickle


## now lets load up the pickle and the labels ##

In [152]:
testIdx= 3
featuresIdxDirFileLoc = "/".join((symbolFeaturesLocation, symbolFeaturesDatesList[testIdx] \
                               ,  commonDatesFeatureDateFile))
featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='iso-8859-1')
#encoding='iso-8859-1' or encoding ='latin1' or 'bytes'

### pick up the features ###

In [155]:

dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                 featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)

In [161]:
dfFeatures.shape

(3599, 21)

In [162]:
labelsCommonDateFile = ".".join((commonDates[commonIdx],'csv'))

labelsCommonFileLoc = "/".join((symbolLabelsLocation, labelsCommonDateFile))
labelsDf=pd.read_csv(labelsCommonFileLoc)

In [168]:
#take the labels out
labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 

labelsShift = labels.isna().sum() # going to use this for "allignment of features and labels"


4

### put labels and features together before you dropnans ###

In [176]:

dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])

In [178]:
dfXY.head(4)

,fischer_score_dlambda,fischer_score_dsigma,fischer_score_dweight,lambda_lambda,lambda_sigma,lambda_weight,sigma_sigma,sigma_weight,weight_weight,gamma_0,...,ksi_0_to_0,ksi_0_to_1,ksi_0_to_2,ksi_1_to_0,ksi_1_to_1,ksi_1_to_2,ksi_2_to_0,ksi_2_to_1,ksi_2_to_2,label_PrMov__window_5__thres_arbitrary__0.1
0,0.000003,0.000000,1.515550,-0.000031,0.000000,0.000004,0.000000e+00,0.000000,2.296892,9.999996e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,2.616586,-934.088591,-1.132378,-36.553836,14.286524,-0.607832,-9.769922e+06,-8.119140,-9.308414,2.220446e-16,...,2.220446e-16,9.978199e-01,2.179722e-03,2.220446e-16,3.813301e-07,7.338355e-10,2.220446e-16,2.000702e-08,1.230780e-10,0.0
2,8.370264,-934.287746,0.471146,-82.878651,11.451600,-0.919716,-9.778087e+06,-6.024955,-11.805625,2.220446e-16,...,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,9.658294e-01,3.199084e-02,2.220446e-16,1.971026e-03,2.086978e-04,0.0
3,14.480640,-934.300571,2.073181,-124.683190,11.042120,-1.272411,-9.778613e+06,-5.883499,-14.366444,5.533573e-07,...,2.220446e-16,2.220446e-16,2.220446e-16,5.091251e-07,9.402840e-01,2.751592e-02,4.423217e-08,2.944503e-02,2.754465e-03,0.0


## get working now on features and labels for MKL ##

In [182]:
dfX = dfXY.drop(columns=[ label_name])
print("Shape of dfX..",dfX.shape[0])    

Shape of dfX.. 3595


## split the data ##

In [186]:
X =dfX
y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Simple Classifiers for Comparison ##

In [ ]:
#clf.fit(X_train, y_train)
classifier = SVC(kernel='linear', C=0.01)
fitd_clf=classifier.fit(X_train, y_train)
# Create a SVC classifier using an RBF kernel
rbf_classifier= SVC(kernel='rbf', random_state=0, gamma=1000, C=1000)
rbf_clf=rbf_classifier.fit(X_train, y_train)